# 深層学習体験会: テキスト音声合成（TTS）

このノートブックでは、**TTS（Text-to-Speech）**技術を使って、テキストから音声を生成する方法を学びます。

## TTSとは？

TTS（Text-to-Speech）とは、テキストを読み上げる音声を自動生成する技術です。このノートブックでは、**Zonos**という高品質なTTSモデルを使用します。

### Zonosの特徴
- 多言語対応（日本語、英語など約100言語）
- 感情表現が可能（喜び、悲しみ、怒りなど）
- 声のクローニング（参照音声から声質を学習）
- ピッチ（音の高さ）と速度の調整が可能

### 応用例
- オーディオブックの作成
- ナビゲーションシステム
- アクセシビリティツール（視覚障害者向け）
- YouTubeのナレーション
- ゲームキャラクターの音声

## このノートブックの流れ
1. 環境の準備とモデルの読み込み
2. 参照音声のアップロード（オプション）
3. テキストと感情設定
4. 音声の生成と再生

## ステップ1: 環境の準備とモデルの読み込み

必要なライブラリをインストールし、Zonosモデルを読み込みます。

In [ ]:
# --- 環境のセットアップとモデルの読み込み ---

# システムの更新とespeak-ngのインストール（音声合成エンジン）
!apt update && apt install -y espeak-ng

# ZonosのGitHubリポジトリをクローン
!git clone https://github.com/Isi-dev/Zonos.git

# Zonosディレクトリに移動
%cd Zonos

# Zonosをインストール（-e: 編集可能モードでインストール）
!pip install -e .
# !pip install --no-build-isolation -e .[compile] # オプション: ハイブリッドモードで実行する場合に必要

# 必要なライブラリをインポート
import torch  # PyTorchフレームワーク
import torchaudio  # 音声処理用ライブラリ
from zonos.model import Zonos  # Zonosモデル
from zonos.conditioning import make_cond_dict  # 条件付け用の関数

# --- デバイスの確認 ---
# CUDAが利用可能ならGPU、そうでなければCPUを使用
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"使用デバイス: {device}")

# --- モデルの読み込み ---
print("モデルを読み込み中...")
# Hugging Faceから事前学習済みモデルをダウンロード
model = Zonos.from_pretrained("Isi99999/Zonos-v0.1-transformer", device=device)
print("モデルの読み込みが完了しました！")

## ステップ2: 参照音声のアップロード（オプション）

このステップは**オプション**です。参照音声をアップロードすると、その声質を学習してクローンすることができます。

### 参照音声の要件
- **長さ**: 10秒〜30秒
- **形式**: MP3など（自動的に処理されます）
- **品質**: クリアで雑音の少ない音声を推奨

スキップする場合は、デフォルトの声が使用されます。

In [ ]:
# --- 参照音声のアップロードと処理 ---

# 必要なライブラリをインポート
import os
from google.colab import files

# 環境変数を設定（UTF-8エンコーディング）
os.environ["LC_ALL"] = "C.UTF-8"
os.environ["LANG"] = "C.UTF-8"

# assetsディレクトリを作成（既に存在する場合はエラーにしない）
os.makedirs("assets", exist_ok=True)

# ファイルアップロードのUIを表示
uploaded = files.upload()

# アップロードされたファイルを処理
for filename in uploaded.keys():
    # 保存先のパス
    new_path = "assets/reference.mp3"
    
    # 既に同名ファイルが存在する場合は削除
    if os.path.exists(new_path):
        os.remove(new_path)
    
    # アップロードされたファイルを安全にリネーム
    os.rename(filename, new_path)

# --- 参照音声の読み込み ---
print("参照音声を読み込み中...")
# 音声ファイルを読み込む（wavとサンプリングレートを取得）
wav, sampling_rate = torchaudio.load("assets/reference.mp3")

# モデルで音声埋め込みベクトルを作成（声の特徴を抽出）
speaker = model.make_speaker_embedding(wav, sampling_rate)
print("参照音声の読み込みが完了しました！")

## ステップ3: テキスト入力と感情設定

ここでメインの音声合成を実行します。様々なパラメータを調整できます。

### 調整可能なパラメータ

1. **text**: 読み上げたいテキスト（日本語にも対応）
2. **seed**: ランダムシード（同じ値で同じ結果を再現可能）
3. **use_default_speaker**: デフォルトの声を使うか（参照音声を使わない場合はTrue）
4. **language**: 言語設定（日本語は'ja'、英語は'en-us'）

#### 感情パラメータ（0.0〜1.0の範囲）
- **happy**: 喜び
- **sad**: 悲しみ
- **disgust**: 嫌悪
- **fear**: 恐怖
- **surprise**: 驚き
- **anger**: 怒り
- **neutral**: 中立
- **other**: その他

**注意**: 感情パラメータの合計は自動的に1.0に正規化されます。

#### 音声パラメータ
- **pitch**: ピッチ（音の高さ）0〜400
- **speed**: 話す速度 0〜40（15が標準）

### 実行方法
1. パラメータを調整
2. セルを実行
3. 音声が生成され、自動的に再生されます

In [ ]:
# --- パラメータ設定 ---
# 読み上げるテキスト（英語のサンプル文）
text = " I am motivated by the simple yet profound joys of being alive—the taste of a good meal, the laughter of a friend, the beauty of a sunrise, and the endless pursuit of knowledge. Even if everything about me ceases when I die, my actions, words, and ideas can leave ripples in the world, affecting others in ways I may never fully grasp. " # @param {type:"string"}

# ランダムシード（同じ値で同じ結果を再現）
seed = 421 # @param {"type":"number"}

# デフォルトの声を使用するかどうか
use_default_speaker = False  # @param {type:"boolean"}

# 言語設定（'en-us': 英語, 'ja': 日本語）
language = 'en-us' # @param ['af', 'am', 'an', 'ar', 'as', 'az', 'ba', 'bg', 'bn', 'bpy', 'bs', 'ca', 'cmn', 'cs', 'cy', 'da', 'de', 'el', 'en-029', 'en-gb', 'en-gb-scotland', 'en-gb-x-gbclan', 'en-gb-x-gbcwmd', 'en-gb-x-rp', 'en-us', 'eo', 'es', 'es-419', 'et', 'eu', 'fa', 'fa-latn', 'fi', 'fr-be', 'fr-ch', 'fr-fr', 'ga', 'gd', 'gn', 'grc', 'gu', 'hak', 'hi', 'hr', 'ht', 'hu', 'hy', 'hyw', 'ia', 'id', 'is', 'it', 'ja', 'jbo', 'ka', 'kk', 'kl', 'kn', 'ko', 'kok', 'ku', 'ky', 'la', 'lfn', 'lt', 'lv', 'mi', 'mk', 'ml', 'mr', 'ms', 'mt', 'my', 'nb', 'nci', 'ne', 'nl', 'om', 'or', 'pa', 'pap', 'pl', 'pt', 'pt-br', 'py', 'quc', 'ro', 'ru', 'ru-lv', 'sd', 'shn', 'si', 'sk', 'sl', 'sq', 'sr', 'sv', 'sw', 'ta', 'te', 'tn', 'tr', 'tt', 'ur', 'uz', 'vi', 'vi-vn-x-central', 'vi-vn-x-south', 'yue']

# --- 感情パラメータ（0.0〜1.0の範囲） ---
happy = 0.3077 # @param {type:"slider", min:0.0, max:1.0, step:0.05} # 喜び
sad = 0.0256 # @param {type:"slider", min:0.0, max:1.0, step:0.05} # 悲しみ
disgust = 0.0256 # @param {type:"slider", min:0.0, max:1.0, step:0.05} # 嫌悪
fear = 0.0256 # @param {type:"slider", min:0.0, max:1.0, step:0.05} # 恐怖
surprise = 0.0256 # @param {type:"slider", min:0.0, max:1.0, step:0.05} # 驚き
anger = 0.0256 # @param {type:"slider", min:0.0, max:1.0, step:0.05} # 怒り
other = 0.2564 # @param {type:"slider", min:0.0, max:1.0, step:0.05} # その他
neutral = 0.3077 # @param {type:"slider", min:0.0, max:1.0, step:0.05} # 中立

# --- 音声パラメータ ---
pitch = 20 # @param {type:"slider", min:0, max:400, step:1} # ピッチ（音の高さ）
speed = 15 # @param {type:"slider", min:0.0, max:40.0, step:1.0} # 話す速度

# --- 感情パラメータの正規化 ---
# 全ての感情値の合計を計算
total = happy + sad + disgust + fear + surprise + anger + other + neutral
if total > 0:
    # 合計が1.0になるように正規化
    happy = happy / total
    sad = sad / total
    disgust = disgust / total
    fear = fear / total
    surprise = surprise / total
    anger = anger / total
    other = other / total
    neutral = neutral / total

# 感情値をテンソルに変換
emotions = torch.tensor(list(map(float, [happy, sad, disgust, fear, surprise, anger, other, neutral])), device=device)

# --- デフォルトの声を使用する場合 ---
if use_default_speaker:
    print("デフォルトの音声を読み込み中...")
    # デフォルトの音声ファイルを読み込む
    wav, sampling_rate = torchaudio.load("assets/exampleaudio.mp3")
    # 音声埋め込みベクトルを作成
    speaker = model.make_speaker_embedding(wav, sampling_rate)
    print("デフォルトの音声を読み込みました！")


def generate_speech2(text, seed=421, language="en-us", emotion_tensor=torch.tensor(list(map(float, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0])), device=device), pitch=20, speed=15):
    """
    テキストから音声を生成する関数
    
    引数:
        text: 読み上げるテキスト
        seed: ランダムシード
        language: 言語コード
        emotion_tensor: 感情パラメータのテンソル
        pitch: ピッチ（音の高さ）
        speed: 話す速度
    
    戻り値:
        生成された音声ファイルのパス
    """
    print(f"音声を生成中: {text}")

    # ランダムシードの設定
    if seed >= 0:
        torch.manual_seed(seed)  # 再現性のためにシードを固定
    else:
        torch.random.seed()  # ランダムなシードを使用

    # --- 条件付けの作成 ---
    # テキスト、言語、声、感情などの条件を辞書にまとめる
    cond_dict = make_cond_dict(
        text=text,  # 読み上げるテキスト
        language=language,  # 言語
        speaker=speaker,  # 声の特徴
        emotion=emotion_tensor,  # 感情パラメータ
        pitch_std=pitch,  # ピッチの標準偏差
        speaking_rate=speed  # 話す速度
    )
    # モデルが理解できる形式に変換
    conditioning = model.prepare_conditioning(cond_dict)

    # --- 音声の生成 ---
    # モデルで音声コードを生成
    codes = model.generate(conditioning)
    # コードを音声波形にデコード（CPUに転送）
    wavs = model.autoencoder.decode(codes).cpu()

    # --- 音声の保存 ---
    filename = "output.wav"
    # WAVファイルとして保存
    torchaudio.save(filename, wavs[0], model.autoencoder.sampling_rate)
    return filename

# --- 音声生成の実行 ---
output_file = generate_speech2(
    text, 
    seed=seed, 
    language=language, 
    emotion_tensor=emotions, 
    pitch=pitch, 
    speed=speed
)

# --- 音声の再生 ---
from IPython.display import Audio
# Jupyter Notebookで音声を再生
Audio(output_file)